# Standard Name Convention

The "Standard Name Convention" is one realization of a convention promoted by the toolbox. It is based on the idea, that every dataset must have a physical unit (or none if it is dimensionless) and that datasets must be identifiable via an identifier attribute rather than the dataset name itself.

The key standard attributes are 

 - `standard_name`: A human- and machine-readable dataset identifier based on construction rules and listed in a "Standard Name Table",
 - `standard_name_table`: List of `standard_name` together with the base unit (SI) and a comprehensive description. It also includes additional information about how a `standard_name` can be transformed into a new `standard_name`
 - `units`: The unit attribute of a dataset. Must not be SI-unit, but must be convertible to it and then match the registered SI-unit in the Standard name table,
 - `long_name`: An alternative name if no `standard_name` is applicable.

This concept is first introduced by the Climate and Forecast community and is called [CF-convention](http://cfconventions.org/). The `h5RDMtoolbox` adopts the concept and implements a general version of it, so that users can define their own discipline- or problem-specific standard name convention.

Main benefits of the convention are:
- achieving self-describing files, which are human and machine interpretation interpretable,
- validating correctness of dataset identifiers (standard_name) and their units
- allowing unit-aware processing of data.

This chapter walks you through the concept and shows how to apply it

In [1]:
import h5rdmtoolbox as h5tbx
import warnings
warnings.filterwarnings('ignore')

from h5rdmtoolbox.convention.standard_names.table import StandardNameTable

## Standard Name Tables

### Example 1: cf-convention

The Standard name table should be defined in documents (typically XML or YAML). The corresponding object then can be initialized by the respective constructor methods (`from_yaml`, `from_web`, ...).

For reading the original CF-convention table, do the following:

In [2]:
cf = StandardNameTable.from_web("https://cfconventions.org/Data/cf-standard-names/79/src/cf-standard-name-table.xml",
                               known_hash='4c29b5ad70f6416ad2c35981ca0f9cdebf8aab901de5b7e826a940cf06f9bae4')
cf

<StandardNameTable: (name: cf-standard-name-table.xml, version_number: 79, last_modified: 2022-03-19T15:25:54Z, institution: Centre for Environmental Data Analysis, contact: support@ceda.ac.uk, version: v79, url: https://cfconventions.org/Data/cf-standard-names/79/src/cf-standard-name-table.xml)>

The standard names are items of the table object:

In [3]:
cf['x_wind']

<StandardName: "x_wind" [m/s] "x" indicates a vector component along the grid x-axis, positive with increasing x. Wind is defined as a two-dimensional (horizontal) air velocity vector, with no vertical component. (Vertical motion in the atmosphere has the standard name upward_air_velocity.).>

In [4]:
cf['x_wind'].units

<Unit('meter / second')>

In [5]:
cf['x_wind'].description

'"x" indicates a vector component along the grid x-axis, positive with increasing x. Wind is defined as a two-dimensional (horizontal) air velocity vector, with no vertical component. (Vertical motion in the atmosphere has the standard name upward_air_velocity.).'

## Example 2: User defined table

Initializing standard name tables from a web-resource should be the standard process, because a project or community might defined it and published it under a DOI.

The `h5rdmtoolbox` especially supports tables that are published on [Zenodo](https://zenodo.org/):

In [6]:
snt = StandardNameTable.from_zenodo(8276716)
snt

<StandardNameTable: (institution: my_institution, contact: https://orcid.org/0000-0001-8729-0482, valid_characters: [^a-zA-Z0-9_], pattern: ^[0-9 ].*, last_modified: 2023-08-16_11:05:00, version: v1.1, zenodo_doi: 10.5281/zenodo.8276716)>

Here are the standard names of the table:

In [7]:
snt.names

['coordinate', 'static_pressure', 'time', 'velocity']

In a notebook, we can also get a nice overview of the table by calling `dump()`:

In [8]:
snt.dump()

,units,description,vector
coordinate,m,Coordinate refers to the spatial coordinate. Coordinate is a vector quantity.,True
static_pressure,Pa,Static pressure refers to the force per unit area exerted by a fluid. Pressure is a scalar quantity.,NaN
time,s,Time refers to the relative time since start of data acquisition.,NaN
velocity,m/s,Velocity refers to the change of position over time. Velocity is a vector quantity.,True


### Transformation of base standard names
Not all allowed standard names must be included in the table. There are some so-called transformations of the listed ones. 
There are two ways to transform a standard name.

 1. Using affixes: Adding a prefix or a suffix
 2. Apply a mathematical operation to the name

#### 1. Adding affixes

Note, that 'x_velocity' is not part of the table:

In [9]:
'x_velocity' in snt

False

... but 'velocity' is. And it is a vector. The vector property tells us, if we can add a "vector component name" as a prefix, e.g. a "x" or "y":

In [10]:
snt['velocity'].is_vector()

True

Which vector component exist, are defined in the table:

In [11]:
snt.affixes['component'].values

{'x': 'X indicates the x-axis component of the vector.',
 'y': 'Y indicates the y-axis component of the vector.',
 'z': 'Z indicates the z-axis component of the vector.'}

Thus, by indexing "x_velocity" the table checks whether the prefix is valid and if yes returns the new (transformed) standard name:

In [12]:
snt['x_velocity']

<StandardName: "x_velocity" [m/s] Velocity refers to the change of position over time. Velocity is a vector quantity. X indicates the x-axis component of the vector.>

#### Apply a mathematical operation

During processing of data, often times datasets are transformed in with mathematical function like taking the square or applying a derivative of one quantity with respect to (wrt) another one. Some mathemtaical operations like these are supported in the version, e.g.:

In [13]:
snt['derivative_of_x_velocity_wrt_x_coordinate']

<StandardName: "derivative_of_x_velocity_wrt_x_coordinate" [1/s] Derivative of x_velocity with respect to x_coordinate. Velocity refers to the change of position over time. Velocity is a vector quantity. X indicates the x-axis component of the vector. Coordinate refers to the spatial coordinate. Coordinate is a vector quantity. X indicates the x-axis component of the vector.>

In [14]:
snt['square_of_static_pressure']

<StandardName: "square_of_static_pressure" [Pa**2] Square of static_pressure. Static pressure refers to the force per unit area exerted by a fluid. Pressure is a scalar quantity.>

In [15]:
snt['arithmetic_mean_of_static_pressure']

<StandardName: "arithmetic_mean_of_static_pressure" [Pa] Arithmetic mean of static_pressure. Static pressure refers to the force per unit area exerted by a fluid. Pressure is a scalar quantity.>

## Usage with HDF5 files

Let's apply the convention to HDF5 files. We lazyly take the existing tutorial convention and remove some standard attributes in order to limit the example to the relevant attributes of the standard name convention:

In [16]:
zenodo_cv = h5tbx.convention.from_zenodo('https://zenodo.org/record/8357399')
sn_cv = zenodo_cv.pop('contact', 'comment', 'references', 'data_type')
sn_cv.name = 'standard name convention'
sn_cv.register()

h5tbx.use(sn_cv)
sn_cv

Convention "h5rdmtoolbox-tutorial-convention" filename: C:\Users\da4323\AppData\Local\h5rdmtoolbox\h5rdmtoolbox\cache\10.5281_zenodo.8357399
creating directory for the convention
No special functions defined
Found 0 function(s) in C:\Users\da4323\AppData\Local\h5rdmtoolbox\h5rdmtoolbox\conventions\h5rdmtoolbox_tutorial_convention\h5rdmtoolbox_tutorial_convention.py


Convention("standard name convention")
contact: https://orcid.org/0000-0001-8729-0482
  File.__init__():
    * standard_name_table (default=<h5rdmtoolbox.convention.consts.DefaultValue object at 0x000001CBD2CD2100>):
		The standard name table of the convention.
  Group.create_dataset():
    * scale (default=None):
		Scale factor for the dataset values.
    * offset (default=None):
		The offset of the dataset.
    * units (default=None):
		The physical unit of the dataset. If dimensionless, the unit is ''.
    * standard_name (default=None):
		Standard name of the dataset. If not set, the long_name attribute must be given.
    * long_name (default=None):
		An comprehensive description of the dataset. If not set, the standard_name attribute must be given.
  Group.create_group(): (Nothing registered)

Find out about the available standard names: We do this by creating a file and retrieving the attribute`standard_name_table`. Based on the convention, it is set by default, so it is available without explicitly setting it:

In [17]:
with h5tbx.File() as h5:
    snt = (h5.standard_name_table)

print('The available (base) standard names are: ', snt.names)

The available (base) standard names are:  ['coordinate', 'static_pressure', 'time', 'velocity']


One possible dataset based on the standard name table could be "x_velocity". This is possible, because *component* is available in the list of **affixes**. Based on the transformation pattern, it is clear the "component" is a **prefix**. "x" is within the available components, so "x_velocity" is a valid transformed standard name from the given table:

In [18]:
print('Available affixes: ', snt.affixes.keys())

print('\nValues for the component prefix:')
snt.affixes['component']

Available affixes:  dict_keys(['device', 'location', 'reference_frame', 'component'])

Values for the component prefix:


<Affix: name="component", description="Components are prefixes to the standard_name, e.g. x_velocity." transformation_pattern=^(.*)_(.*)$, values=['x', 'y', 'z']>

Let's access the name from the table. It exists and the description is adjusted, too:

In [19]:
snt['x_velocity']

<StandardName: "x_velocity" [m/s] Velocity refers to the change of position over time. Velocity is a vector quantity. X indicates the x-axis component of the vector.>

Creating a x-velocity dataset:

In [20]:
with h5tbx.File() as h5:
    h5.create_dataset('u', data=[1,2,3], standard_name='x_velocity', units='km/s')
    h5.dump()